In [64]:
#Mengimpor dataset dari penyimpanan
import weka.core.jvm as jvm
from weka.core.converters import Loader
jvm.start() 
from weka.classifiers import Evaluation
from weka.classifiers import Random
import pandas as pd
import numpy as np
load = Loader(classname="weka.core.converters.ArffLoader")
anomali = load.load_file('dataset1.arff')
df = pd.DataFrame(anomali)
backup_time = np.array(df.iloc[:,1])


INFO:weka.core.jvm:JVM already running, call jvm.stop() first


In [65]:
anomali
anomali.no_class()

In [66]:
anomali

@relation dataset1

@attribute id numeric
@attribute time {'09:19:33 19/05/1995','09:19:34 19/05/1995','09:19:35 19/05/1995','09:19:36 19/05/1995','09:19:37 19/05/1995','09:19:38 19/05/1995','09:19:39 19/05/1995','09:19:40 19/05/1995','09:19:41 19/05/1995','09:19:42 19/05/1995','09:19:43 19/05/1995','09:19:44 19/05/1995','09:19:46 19/05/1995','09:19:47 19/05/1995','09:19:48 19/05/1995','09:19:49 19/05/1995','09:19:50 19/05/1995','09:19:51 19/05/1995','09:19:52 19/05/1995','09:19:53 19/05/1995','09:19:54 19/05/1995','09:19:55 19/05/1995','09:19:56 19/05/1995','09:19:57 19/05/1995','09:19:58 19/05/1995','09:19:59 19/05/1995','09:20:00 19/05/1995','09:20:01 19/05/1995','09:20:02 19/05/1995','09:20:03 19/05/1995','09:20:04 19/05/1995','09:20:05 19/05/1995','09:20:06 19/05/1995','09:20:07 19/05/1995','09:20:08 19/05/1995','09:20:09 19/05/1995','09:20:10 19/05/1995','09:20:11 19/05/1995','09:20:12 19/05/1995','09:20:13 19/05/1995','09:20:14 19/05/1995','09:20:15 19/05/1995','09:20:16 19/05/1

In [57]:
#Pre-process
data_test = []
n = input('window size : ')
rec = n
size = anomali.num_instances
backup_class = anomali.values(anomali.num_attributes-1)
anomali.delete_attribute(anomali.num_attributes-1)
anomali.delete_first_attribute()
anomali.delete_first_attribute()
n = int(n)
for x in range(n):
    y = str(anomali.get_instance(x)).split(',')
    data_test.append(y)
    anomali.delete(0)
names = []
data_inst = [] 
data_pred = [] 
data_std = []
anomali_out = [] 
anomali_per_baris = []
n_attr = anomali.num_attributes
row = anomali.num_instances
n_inst = (anomali.num_instances) * n_attr
for x in range(n_attr):
    names.append(anomali.attribute(x))
anomali.class_is_first()

window size : 30


In [58]:
#In-process
def proses():
    import math
    from weka.classifiers import Kernel, KernelClassifier
    from weka.classifiers import PredictionOutput
    import numpy as np
    klasifi = KernelClassifier(classname="weka.classifiers.functions.SMOreg", options=["-N","0"])
    vm = Kernel(classname="weka.classifiers.functions.supportVector.RBFKernel", options=["-G","0.1"])
    klasifi.vm = vm
    output_x = PredictionOutput(classname="weka.classifiers.evaluation.output.prediction.PlainText")
    kelola = Evaluation(anomali)
    kelola.crossvalidate_model(klasifi,anomali, 10, Random(0), output=output_x)
    process = 0
    for x in anomali.values(anomali.class_index):
        data_inst.append(x)
    for x in kelola.predictions:
        i = str(x)
        index = i.split()
        data_pred.append(float(index[2]))
    data_std.insert(idx,math.ceil(np.std(data_inst)))
    print ('\n DONE PROCESSING DATASET ATTRIBUTE ', anomali.attribute(anomali.class_index),'...')

In [59]:
# menginisiasi disetiap value pada attribute tertentu apakah anomali atau tidak(dibandingkan dgn std)
def inisiasi_anomali():
    print (len(data_pred))
    baris = 0
    start = 0
    for ite in range(n_attr):
        for inisiasi in range(row):
            if ite == 0:
                err = abs(data_pred[inisiasi]-data_inst[inisiasi]) #menghitung nilai error
                if err > data_std[ite]:
                    anomali_out.append(1)
                else:
                    anomali_out.append(0)
                baris = inisiasi
                start = baris
            else:
                err = abs(data_pred[baris]-data_inst[baris])
                if err > data_std[ite]:
                    anomali_out.append(1)
                else:
                    anomali_out.append(0)
                baris += 1
        start = baris
        baris = start
def view():
    start = 0
    an = [] #untuk menampung values yang anomali 
    data_anomali = [] #untuk menampung atribute yg anomali 
    record = int(rec)
    #record = rec
    for n in range(row):
        if n == 0:
            print ('\nTransformatting.....')
            print ('Record ke - ', record,' ', data_inst[0],', ',data_inst[row*1],', ',data_inst[row*2],', ',data_inst[row*3],', ',data_inst[row*4])
            print ('PREDICT : ', data_pred[0],', ',data_pred[row*1],', ',data_pred[row*2],', ',data_pred[row*3],', ',data_pred[row*4])
            print ('STD     : ', data_std[0:5])
            count_an = 0
            count_not = 0
            data_anomali = []
            for y in range(n_attr):
                if anomali_out[start] == 1:
                    data_anomali.append(names[y])
                    count_an += 1
                    start += 1
                else:
                    count_not += 1
                    start += 1
            print ('An anomali : ', data_anomali)
            if count_an > count_not:
                anomali_per_baris.append(1)
                an.append(record)
            else:
                anomali_per_baris.append(0)
            print ('\n-------PEMISAH---------\n')
        else:
            print ('\nTransformatting.....')
            print ('Record ke - ', record+1,' ', data_inst[0+n],', ',data_inst[(row*1)+n],', ',data_inst[(row*2)+n],', ',data_inst[(row*3)+n],', ',data_inst[(row*4)+n])
            print ('PREDICT : ', data_pred[0+n],', ',data_pred[(row*1)+n],', ',data_pred[(row*2)+n],', ',data_pred[(row*3)+n],', ',data_pred[(row*4)+n])
            print ('STD     : ', data_std[0:5])
            data_anomali = []
            count_an = 0 
            count_not = 0
            for y in range(n_attr):
                if anomali_out[start] == 1:
                    data_anomali.append(names[y])
                    count_an += 1
                    start += 1
                else:
                    count_not += 1
                    start += 1
            print ('An anomali : ', data_anomali)
            if count_an > count_not:
                anomali_per_baris.append(1)
                an.append(record)
            else:
                anomali_per_baris.append(0)
            print ('\n-------PEMISAH---------\n')
            record += 1
    for x in an:
        print (backup_time[x],' Record of ',x,' is an anomaly')
    print ('Count : ', len(an))
for idx in range(n_attr):
    proses()
    anomali.no_class()
    anomali.delete_first_attribute()
    if idx == 4:
        break
    else:
        anomali.class_is_first()
inisiasi_anomali()
view()


 DONE PROCESSING DATASET ATTRIBUTE  @attribute ABPmean numeric ...

 DONE PROCESSING DATASET ATTRIBUTE  @attribute HR numeric ...

 DONE PROCESSING DATASET ATTRIBUTE  @attribute PULSE numeric ...

 DONE PROCESSING DATASET ATTRIBUTE  @attribute RESP numeric ...

 DONE PROCESSING DATASET ATTRIBUTE  @attribute SpO2 numeric ...
1355

Transformatting.....
Record ke -  30   110.0 ,  101.0 ,  100.0 ,  21.0 ,  99.0
PREDICT :  109.02458538717907 ,  95.14024346764096 ,  93.13101451787796 ,  21.964720000000085 ,  50.00000000000004
STD     :  [2, 8, 26, 39, 39]
An anomali :  [@attribute PULSE numeric, @attribute RESP numeric]

-------PEMISAH---------


Transformatting.....
Record ke -  31   109.0 ,  101.0 ,  99.0 ,  21.0 ,  99.0
PREDICT :  110.16613296566108 ,  95.97412506239127 ,  -0.00337721751589906 ,  20.82400000000019 ,  50.00000000000004
STD     :  [2, 8, 26, 39, 39]
An anomali :  []

-------PEMISAH---------


Transformatting.....
Record ke -  32   108.0 ,  101.0 ,  98.0 ,  21.0 ,  99.0
PRE

Record ke -  88   108.0 ,  98.0 ,  0.0 ,  26.0 ,  0.0
PREDICT :  108.73163157298748 ,  95.03782319656197 ,  95.09637929898844 ,  21.999999999999773 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  []

-------PEMISAH---------


Transformatting.....
Record ke -  89   109.0 ,  99.0 ,  0.0 ,  25.0 ,  0.0
PREDICT :  109.23969118548732 ,  94.99852374071534 ,  95.1370613448317 ,  21.999999999999773 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  [@attribute SpO2 numeric]

-------PEMISAH---------


Transformatting.....
Record ke -  90   109.0 ,  99.0 ,  0.0 ,  24.0 ,  0.0
PREDICT :  109.23969118548732 ,  94.99852374071534 ,  95.1370613448317 ,  21.999999999999773 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  []

-------PEMISAH---------


Transformatting.....
Record ke -  91   109.0 ,  101.0 ,  0.0 ,  23.0 ,  0.0
PREDICT :  111.14244460769875 ,  95.93025864401473 ,  -0.020341022921632715 ,  20.000000000000078 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  []

-------PEMISAH---

STD     :  [2, 8, 26, 39, 39]
An anomali :  [@attribute SpO2 numeric]

-------PEMISAH---------


Transformatting.....
Record ke -  145   112.0 ,  93.0 ,  0.0 ,  25.0 ,  0.0
PREDICT :  108.45232120321211 ,  95.07435954765413 ,  96.89541007975289 ,  21.923204081632555 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  [@attribute ABPmean numeric, @attribute HR numeric, @attribute RESP numeric]

-------PEMISAH---------


Transformatting.....
Record ke -  146   113.0 ,  93.0 ,  0.0 ,  25.0 ,  100.0
PREDICT :  111.06150340479333 ,  95.8895841141663 ,  -0.17490780271958942 ,  20.902795918367037 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  [@attribute SpO2 numeric]

-------PEMISAH---------


Transformatting.....
Record ke -  147   113.0 ,  93.0 ,  96.0 ,  25.0 ,  100.0
PREDICT :  108.07066053824212 ,  95.13446399257917 ,  95.9579393834449 ,  21.9129999999999 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  []

-------PEMISAH---------


Transformatting.....
Record ke -  148   113.0 

Record ke -  199   108.0 ,  96.0 ,  97.0 ,  27.0 ,  98.0
PREDICT :  109.20000447179817 ,  95.00567317953316 ,  95.80454533292153 ,  21.912000000000084 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  []

-------PEMISAH---------


Transformatting.....
Record ke -  200   108.0 ,  95.0 ,  97.0 ,  27.0 ,  98.0
PREDICT :  108.17571337801371 ,  95.13816815649797 ,  96.8212905930614 ,  21.922204081632742 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  []

-------PEMISAH---------


Transformatting.....
Record ke -  201   108.0 ,  95.0 ,  97.0 ,  27.0 ,  98.0
PREDICT :  109.11021768062834 ,  95.01134635906628 ,  96.73955403496034 ,  21.922204081632742 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  []

-------PEMISAH---------


Transformatting.....
Record ke -  202   107.0 ,  95.0 ,  97.0 ,  99.0 ,  97.0
PREDICT :  108.99968705356234 ,  95.04849499567922 ,  94.8368420076423 ,  21.901795918367434 ,  50.0
STD     :  [2, 8, 26, 39, 39]
An anomali :  []

-------PEMISAH---------


Transfo

An anomali :  [@attribute ABPmean numeric, @attribute HR numeric, @attribute PULSE numeric, @attribute RESP numeric, @attribute SpO2 numeric]

-------PEMISAH---------


Transformatting.....
Record ke -  251   109.0 ,  95.0 ,  95.0 ,  23.0 ,  97.0
PREDICT :  108.37857459709082 ,  95.08313098225729 ,  95.9378725553302 ,  21.98000000000021 ,  50.00000000000004
STD     :  [2, 8, 26, 39, 39]
An anomali :  [@attribute ABPmean numeric, @attribute HR numeric, @attribute PULSE numeric, @attribute RESP numeric, @attribute SpO2 numeric]

-------PEMISAH---------


Transformatting.....
Record ke -  252   109.0 ,  95.0 ,  95.0 ,  23.0 ,  97.0
PREDICT :  108.86393232857141 ,  95.01088008486452 ,  95.95562022739165 ,  21.98000000000021 ,  50.00000000000004
STD     :  [2, 8, 26, 39, 39]
An anomali :  [@attribute ABPmean numeric, @attribute HR numeric, @attribute PULSE numeric, @attribute RESP numeric, @attribute SpO2 numeric]

-------PEMISAH---------


Transformatting.....
Record ke -  253   110.0 ,  9

09:20:23 19/05/1995  Record of  49  is an anomaly
09:20:24 19/05/1995  Record of  50  is an anomaly
09:20:25 19/05/1995  Record of  51  is an anomaly
09:20:26 19/05/1995  Record of  52  is an anomaly
09:20:56 19/05/1995  Record of  81  is an anomaly
09:20:57 19/05/1995  Record of  82  is an anomaly
09:22:01 19/05/1995  Record of  144  is an anomaly
09:22:06 19/05/1995  Record of  149  is an anomaly
09:22:07 19/05/1995  Record of  150  is an anomaly
09:22:09 19/05/1995  Record of  152  is an anomaly
09:22:12 19/05/1995  Record of  155  is an anomaly
09:22:13 19/05/1995  Record of  156  is an anomaly
09:22:14 19/05/1995  Record of  157  is an anomaly
09:22:15 19/05/1995  Record of  158  is an anomaly
09:22:17 19/05/1995  Record of  160  is an anomaly
09:22:23 19/05/1995  Record of  166  is an anomaly
09:23:25 19/05/1995  Record of  226  is an anomaly
09:23:26 19/05/1995  Record of  227  is an anomaly
09:23:45 19/05/1995  Record of  246  is an anomaly
09:23:46 19/05/1995  Record of  247  

In [60]:
#Processed Machine Learning
import math
predict_true = 0
predict_false = 0
predict = []
data_train = []
start = 0
val = 0
for x in range(n_attr):
    for y in range(row):
        if x == 0:
            klasifikasi = math.ceil((abs(data_inst[y]-data_pred[y])/data_inst[y])*100)
            err = math.ceil(abs(data_inst[y]-data_pred[y]))
            if klasifikasi == err:
                predict_true += 1
            else:
                predict_false += 1
            if predict_true > predict_false:
                predict.append(1)
            else:
                predict.append(0)
            start = y
        else:
            klasifikasi = math.ceil((abs(data_inst[start]-data_pred[start])/data_inst[start])*100)
            err = math.ceil(abs(data_inst[y]-data_pred[y]))
            if klasifikasi == err:
                predict_true += 1
            else:
                predict_false += 1
            if predict_true > predict_false:
                predict.append(1)
            else:
                predict.append(0)
start = 0
error = 0
for x in range(row):
    cout = (str(int(data_inst[0+x]))+' '+str(int(data_inst[(row*1)+x]))+' '+ str(int(data_inst[(row*2)+x]))+' '+ str(int(data_inst[(row*3)+x]))+' '+ str(int(data_inst[(row*4)+x]))).split()
    data_train.append(cout)
for x in range(row):
    counting = 0
    for y in data_test[start]:
        if y in data_train[x]:
            counting += 1
    if counting >= 3:
        if anomali_per_baris[x] == 1:
            anomali_per_baris.insert(start, 1)
        elif anomali_per_baris[x] == 0:
            anomali_per_baris.insert(start, 0)
        else:
            error += 1
    else:
        val = anomali_per_baris[x]
        anomali_per_baris.insert(start, val)
    start += 1
    if start == n:
        break

In [61]:
#Identify Anomaly for True False(TP), True Negative(TN), False Positive(FP), False Negative(FN)
import math
start = 0
TP = 0
FN = 0
FP = 0
TN = 0
n = 0
for x in range(size):
    if(backup_class[x] == 0) and (anomali_per_baris[x] == 0):
        TN += 1
    elif(backup_class[x] == 1) and (anomali_per_baris[x] == 0) and (predict[x] == 0):
        FN += 1
    elif(backup_class[x] == 0) and (anomali_per_baris[x] == 1) and (predict[x] == 1):
        FP += 1
    elif predict[x] == 1:
        TP += 1 
DR = (float(TP)/(float(TP+FN)))* 100
FPR = (float(FP)/(float(FP+TN)))* 100
print ('True Positive  : ',TP)
print ('False Positive : ',FP-error)
print ('False Negative : ',FN)
print ('True Negative  : ',TN,'\n')
print ('Detection Rate : ',DR)
print ('False Positive Rate : ', FPR)


True Positive  :  61
False Positive :  80
False Negative :  0
True Negative  :  160 

Detection Rate :  100.0
False Positive Rate :  33.33333333333333
